# Diabetic Population: Data Preparation

    Adajar, Beatrice
    Dygico, Jacob
    Lee, Joan

## Profile Data

We will start with reading in the data file and profiling the dataset to get a summary of our data. From there, we will decide on what data cleaning steps are necessary before proceeding to modeling.

In [1]:
import re
import pandas as pd
from collections import Counter
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.over_sampling import SMOTE
from math import floor
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
file_dir = '.'    # replace with your own file directory, if needed
df = pd.read_csv('{}/diabetic_data.csv'.format(file_dir), na_values='?',
                 dtype={'admission_type_id': str, 'discharge_disposition_id': str,
                        'admission_source_id': str})
df.head()

C:\Users\Bea\AppData\Local\Temp\ipykernel_4464\3274348555.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('{}/diabetic_data.csv'.format(file_dir), na_values='?',


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

Number of rows: 101766
Number of columns: 50


In [4]:
# profile = ProfileReport(df, title="Diabetes Data Report")
# profile.to_file("Diabetes Data Report (Initial).html")
# profile.to_widgets()

Based on [the paper](https://www.hindawi.com/journals/bmri/2014/781670/) related to the dataset, we will do the following steps:

- [ ] Keep only first encounter per `patient_nbr`
- [ ] Drop rows of encounters where patient expired or was discharged to a hospice

Based on the **Alerts** section, we will perform the following initial data cleaning steps:

- [ ] Drop columns with identifier values (not relevant to prediction): <span class="label label-info">encounter_id</span> <span class="label label-info">patient_nbr</span>
- [ ] Drop columns with constant values: <span class="label label-info">examide</span> <span class="label label-info">citoglipton</span> <span class="label label-info">glimepiride-pioglitazone</span>
- [ ] Drop columns with more than 50% missing values: <span class="label label-info">weight</span>

Based on the column values, we need to group together similar values. These steps will help reduce the number of columns:

- [ ] Group together values for columns with high cardinality: <span class="label label-warning">diag_1</span> <span class="label label-warning">diag_2</span> <span class="label label-warning">diag_3</span> <span class="label label-warning">medical_specialty</span>
- [ ] Group together `'NULL'`, `'Not Mapped'`, `'Not Available'`, and `'Unknown/Invalid'` values for ID columns: <span class="label label-warning">admission_type_id</span> <span class="label label-warning">discharge_disposition_id</span> <span class="label label-warning">admission_source_id</span>

Finally, we need to do the following to prep the data for our classification problem:

- [ ] Convert categorical columns with meaningful levels to numerical: <span class="label label-success">[all medication columns]</span> <span class="label label-success">age</span> <span class="label label-success">A1Cresult</span> <span class="label label-success">max_glu_serum</span> <span class="label label-success">change</span> <span class="label label-success">diabetesMed</span>
- [ ] Dummify remaining categorical columns: <span class="label label-success">race</span> <span class="label label-success">payer_code</span> <span class="label label-success">medical_specialty</span> <span class="label label-success">gender</span> <span class="label label-success">diag_1</span> <span class="label label-success">diag_2</span> <span class="label label-success">diag_3</span>
- [ ] Group together `'NO'` and `'>30'` values: <span class="label label-success">readmitted</span>

Before performing any of these steps, let us replace the `np.nan` values in our dataframe with the string `'None'`.

In [5]:
df.fillna('None', inplace=True)

## Drop Rows

- [x] Keep only first encounter per `patient_nbr`
- [x] Drop rows of encounters where patient expired or was discharged to a hospice

First encounter per patient is determined by the earliest `encounter_id` per `patient_nbr`.

In [6]:
df = df.sort_values(['patient_nbr', 'encounter_id']).groupby('patient_nbr').first()
print("Number of rows after dropping subsequent encounters per patient:", df.shape[0])
df = df[~df['discharge_disposition_id'].isin(['11', '13', '14', '19', '20', '21'])] 
print("Number of rows after dropping patients that expired/were discharged to hospice:", df.shape[0])

Number of rows after dropping subsequent encounters per patient: 71518
Number of rows after dropping patients that expired/were discharged to hospice: 69973


## Drop Columns

- [x] Drop columns with identifier values (not relevant to prediction): <span class="label label-info">encounter_id</span> <span class="label label-info">patient_nbr</span>
- [x] Drop columns with constant values: <span class="label label-info">examide</span> <span class="label label-info">citoglipton</span>  <span class="label label-info">glimepiride-pioglitazone</span>
- [x] Drop columns with more than 50% missing values: <span class="label label-info">weight</span>

We start with dropping columns from prediction to decrease the dataframe size.

In [7]:
print("Dataframe size before dropping columns:", df.size)
df.drop(columns=['encounter_id', 'examide', 'citoglipton', 'glimepiride-pioglitazone', 'weight'], axis=1, inplace=True)
print("Dataframe size after dropping columns:", df.size)

Dataframe size before dropping columns: 3428677
Dataframe size after dropping columns: 3078812


## Group Similar Values

- [x] Group together values for columns with high cardinality: <span class="label label-warning">diag_1</span> <span class="label label-warning">diag_2</span> <span class="label label-warning">diag_3</span> <span class="label label-warning">medical_specialty</span>

We need to reduce the cardinality of the diagnosis columns. We can do this by grouping ICD-9 codes following [these categories](http://www.icd9data.com/2006/Volume1/default.htm).

The following code will generate new columns <span class="label label-warning">diag_1_group</span>, <span class="label label-warning">diag_2_group</span>, <span class="label label-warning">diag_3_group</span> and drop the original diagnosis columns.

In [8]:
def get_code_group(code):
    """Given a diagnosis code and the appropriate dictionary, find and return
    the corresponding group of that diagnosis code.
    """
    code = floor(float(re.search(r'\d+.*', code).group(0)))
    
# meaning of each ID for each diagnosis code in code_map
#     0: 'Infectious And Parasitic Diseases',
#     1: 'Neoplasms',
#     2: 'Endocrine, Nutritional And Metabolic Diseases, And Immunity Disorders',
#     3: 'Diseases Of The Blood And Blood-Forming Organs',
#     4: 'Mental Disorders',
#     5: 'Diseases Of The Nervous System And Sense Organs',
#     6: 'Diseases Of The Circulatory System',
#     7: 'Diseases Of The Respiratory System',
#     8: 'Diseases Of The Digestive System',
#     9: 'Diseases Of The Genitourinary System',
#     10: 'Complications Of Pregnancy, Childbirth, And The Puerperium',
#     11: 'Diseases Of The Skin And Subcutaneous Tissue',
#     12: 'Diseases Of The Musculoskeletal System And Connective Tissue',
#     13: 'Congenital Anomalies',
#     14: 'Certain Conditions Originating In The Perinatal Period',
#     15: 'Symptoms, Signs, And Ill-Defined Conditions',
#     16: 'Injury And Poisoning',
#     17: 'Supplementary Classification Of Factors Influencing Health Status And Contact With Health Services',
#     18: 'Supplementary Classification Of External Causes Of Injury And Poisoning'

    codes_map = {
        range(1,140): '0',
        range(140,240): '1',
        range(240,280): '2',
        range(280,290): '3',
        range(290,320): '4',
        range(320,390): '5',
        range(390,460): '6',
        range(460,520): '7',
        range(520,580): '8',
        range(580,630): '9',
        range(630,680): '10',
        range(680,710): '11',
        range(710,740): '12',
        range(740,760): '13',
        range(760,780): '14',
        range(780,800): '15',
        range(800,1000): '16'
    }

    for key, value in codes_map.items():
        if code in key:
            return value

def get_group_diagnosis_codes(df, col):
    """Assign the appropriate group for each diagnosis code for a given column of
    diagnosis codes. Returns the updated dataframe.
    """
    col_group = '{}_group'.format(col)
    
    if col_group not in df.columns:
        df[col_group] = 'None'
    
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        if row[col] == 'None':
            continue
        elif row[col].startswith('V'):
            df.loc[idx, col_group] = '17'
        elif row[col].startswith('E'): 
            df.loc[idx, col_group] = '18'
        else:
            df.loc[idx, col_group] = get_code_group(row[col])
    
    return df

In [9]:
diag_cols = ['diag_1', 'diag_2', 'diag_3']

for col in diag_cols:
    df = get_group_diagnosis_codes(df, col)

# drop original diagnosis columns
df.drop(columns=diag_cols, axis=1, inplace=True)

100%|██████████| 69973/69973 [00:33<00:00, 2062.58it/s]


We will also retain only the top 10 most frequent values under `medical_specialty` and group the rest into an `'Others'` value (excluding missing values).

In [10]:
def map_specialties(x):
    top_medical_specialties = ['InternalMedicine', 'Emergency/Trauma', 'Family/GeneralPractice',
                               'Cardiology', 'Surgery-General', 'Nephrology', 'Orthopedics',
                               'Orthopedics-Reconstructive', 'Radiologist', 'Pulmonology', 'None']
    
    if x not in top_medical_specialties:
        return 'Others'
    else:
        return x

df['medical_specialty'] = df['medical_specialty'].apply(map_specialties)
df['medical_specialty'].unique()

array(['Cardiology', 'Others', 'InternalMedicine', 'None',
       'Orthopedics-Reconstructive', 'Surgery-General',
       'Emergency/Trauma', 'Nephrology', 'Family/GeneralPractice',
       'Pulmonology', 'Orthopedics', 'Radiologist'], dtype=object)

- [x] Group together `'NULL'`, `'Not Mapped'`, `'Not Available'`, and `'Unknown/Invalid'` values for ID columns: <span class="label label-warning">admission_type_id</span> <span class="label label-warning">discharge_disposition_id</span> <span class="label label-warning">admission_source_id</span>

We will also be grouping the listed values as they will all be treated as "missing" values.

In [11]:
id_map = {
    'admission_type_id': {'5':'6', '8':'6'},
    'discharge_disposition_id': {'25':'18', '26':'18'},
    'admission_source_id': {'15':'17', '9':'17', '20':'17', '21':'17'}
}

for key, value in id_map.items():
    df[key] = df[key].apply(lambda x: str(value[x]) if x in value.keys() else str(x))

## Modify Columns for Classification

- [x] Convert categorical columns with meaningful levels to numerical: <span class="label label-success">[all medication columns]</span> <span class="label label-success">age</span> <span class="label label-success">A1Cresult</span> <span class="label label-success">max_glu_serum</span> <span class="label label-success">change</span> <span class="label label-success">diabetesMed</span>

In [12]:
# maps of categorical values to numerical values
medication_map = {'No': 0, 'Down': 1, 'Steady': 2, 'Up': 3}
age_map = {'[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35, '[40-50)': 45,
           '[50-60)': 55, '[60-70)': 65, '[70-80)': 75, '[80-90)': 85, '[90-100)': 95}
A1Cresult_map = {'None': 0, 'Norm': 1, '>7': 2, '>8':3}
max_glu_serum_map = {'None': 0, 'Norm': 1, '>200': 2, '>300': 3}
change_map = {'No': 0, 'Ch': 1}
diabetesMed_map = {'No': 0, 'Yes': 1}

# list of columns to map
medication_columns = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
                      'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
                      'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'insulin',
                      'glyburide-metformin', 'glipizide-metformin', 'metformin-rosiglitazone',
                      'metformin-pioglitazone']
columns_to_map = ['age', 'A1Cresult', 'max_glu_serum', 'change', 'diabetesMed']
map_list = [age_map, A1Cresult_map, max_glu_serum_map, change_map, diabetesMed_map]

In [13]:
for idx, col in enumerate(columns_to_map):
    df[col] = df[col].map(map_list[idx])

for col in medication_columns:
    df[col] = df[col].map(medication_map)

- [x] Dummify remaining categorical columns: <span class="label label-success">race</span> <span class="label label-success">payer_code</span> <span class="label label-success">medical_specialty</span> <span class="label label-success">gender</span> <span class="label label-success">diag_1</span> <span class="label label-success">diag_2</span> <span class="label label-success">diag_3</span>

We apply one-hot encoding to columns with categorical values. We will then drop one column per category to obtain `k-1` dummies.

In [14]:
df_final = pd.get_dummies(df.drop(columns='readmitted'))
df_final = pd.merge(df['readmitted'], df_final, left_index=True, right_index=True)
df_final.drop(columns=['race_None', 'payer_code_None', 'medical_specialty_None',
                       'gender_Unknown/Invalid', 'diag_1_group_None', 'diag_2_group_None',
                       'diag_3_group_None'], axis=1, inplace=True)

- [x] Group together `'NO'` and `'>30'` values: <span class="label label-success">readmitted</span>

Finally, we modify the label column to be a binary class problem. Since our concern is readmission within 30 days, we will map `'<30'` to `1` and everything else to `0`.

In [15]:
df_final['readmitted'] = df_final['readmitted'].map({'NO': 0, '>30': 0, '<30': 1})
df_final['readmitted'].value_counts()

0    63696
1     6277
Name: readmitted, dtype: int64

## Save New Dataset

We can also generate a report of the new dataset to view a summary after the transformations we applied.

In [16]:
# profile = ProfileReport(df_final, title="Diabetes Data Report")
# profile.to_file("Diabetes Data Report (Final).html")
# profile.to_widgets()

We will now split the data into train-test files and pickle them for modeling later. Since the dataset is imbalanced (88.8% not readmitted, 11.6% readmitted), we will do a stratified train-test split.

To account for the class imbalance during training, we will use SMOTE to oversample the minority class in the training data.

In [17]:
# split data into train-test
t_size = 0.4
X = df_final.drop(columns=['readmitted'], axis=1)
y = df_final['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size, random_state=0, stratify=df_final['readmitted'])
print("Class distribution before resampling:")
print(sorted(Counter(y_train).items()))

# oversample minority class
smote = SMOTE(sampling_strategy='minority', random_state=2022)
X_train1, y_train1 = smote.fit_resample(X_train, y_train)
print("Class distribution after SMOTE:")
print(sorted(Counter(y_train1).items()))

# mixed oversample minority class + undersample majority class using SMOTETomek
smote = SMOTETomek(random_state=2022)
X_train2, y_train2 = smote.fit_resample(X_train, y_train)
print("Class distribution after SMOTETomek:")
print(sorted(Counter(y_train2).items()))

# mixed oversample minority class + undersample majority class using SMOTEENN
smote = SMOTEENN(random_state=2022)
X_train3, y_train3 = smote.fit_resample(X_train, y_train)
print("Class distribution after SMOTEENN:")
print(sorted(Counter(y_train3).items()))

Class distribution before resampling:
[(0, 38217), (1, 3766)]
Class distribution after SMOTE:
[(0, 38217), (1, 38217)]
Class distribution after SMOTETomek:
[(0, 38186), (1, 38186)]
Class distribution after SMOTEENN:
[(0, 13336), (1, 37964)]


In [18]:
# pickle files
data_dir = './data' # replace with your own file directory, if needed
X_train3.to_pickle('{}/X_train.pkl'.format(data_dir))     # change X_train to variable depending on chosen sampling method
X_test.to_pickle('{}/X_test.pkl'.format(data_dir))
y_train3.to_pickle('{}/y_train.pkl'.format(data_dir))
y_test.to_pickle('{}/y_test.pkl'.format(data_dir))

## Principal Component Analysis

We will also create a separate dataset where we perform PCA and compare the results. Note that the number of components we choose will correspond to 90% of the variance.

In [19]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

df_features = df_final.drop(columns='readmitted')

scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(df_features)

pca = PCA(n_components=0.90)
pca.fit(data_rescaled)
reduced = pca.transform(data_rescaled)

In [20]:
df_pca = pd.DataFrame(reduced)
df_pca.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,1.073826,-0.487091,0.611997,0.123178,0.726414,-0.388446,-0.459743,0.339648,0.284738,0.610278,...,0.129785,-0.185195,0.094374,0.135401,0.318169,0.331017,0.063520,0.653091,-0.124378,0.006021
1,1.247766,-0.845708,-0.808020,-0.630327,-0.422334,-0.697335,0.087451,-0.531083,-0.522133,-0.069375,...,0.067710,-0.075374,0.115559,-0.112575,0.114162,0.171453,-0.150462,0.133061,-0.342664,-0.157979
2,-0.958929,-0.757357,0.003695,0.782760,-0.832384,0.119101,0.134851,-0.359732,0.324111,-0.019120,...,-0.080763,0.062845,-0.098346,0.020104,-0.018690,-0.118191,-0.090133,-0.017387,-0.067647,0.023735
3,-0.878131,-0.346876,0.342805,0.195924,0.737271,-0.121768,-0.681246,0.152436,-0.087569,-0.806423,...,-0.001618,-0.117945,0.036786,0.094410,-0.098926,-0.035077,-0.037398,0.029968,0.000110,-0.071760
4,-0.989832,-0.756759,-0.335043,-1.150972,0.324396,-0.094022,0.683105,-0.129843,0.029809,-0.018579,...,0.272370,0.107550,0.100200,-0.127839,-0.069921,-0.100507,0.104900,0.149092,-0.281492,0.133402


In [21]:
pca.explained_variance_ratio_.round(3)

array([0.093, 0.07 , 0.056, 0.05 , 0.046, 0.039, 0.038, 0.032, 0.028,
       0.025, 0.024, 0.02 , 0.019, 0.019, 0.016, 0.016, 0.015, 0.014,
       0.014, 0.013, 0.012, 0.011, 0.011, 0.01 , 0.01 , 0.01 , 0.009,
       0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.007, 0.007, 0.007,
       0.006, 0.006, 0.006, 0.006, 0.006, 0.006, 0.005, 0.005, 0.005,
       0.005, 0.005, 0.005, 0.005, 0.005, 0.004, 0.004, 0.004, 0.004,
       0.004, 0.004, 0.004, 0.004, 0.004, 0.003])

In [22]:
df_pca.shape

(69973, 60)

## Save New Dataset (PCA)

We will now split the data into train-test files and pickle them for modeling later. Since the dataset is imbalanced (88.8% not readmitted, 11.6% readmitted), we will do a stratified train-test split.

To account for the class imbalance during training, we will use SMOTE to oversample the minority class in the training data.

In [23]:
# split data into train-test
t_size = 0.4
X = df_pca
y = df_final['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size, random_state=0, stratify=df_final['readmitted'])
print("Class distribution before resampling:")
print(sorted(Counter(y_train).items()))

# oversample minority class
smote = SMOTE(sampling_strategy='minority', random_state=2022)
X_train1, y_train1 = smote.fit_resample(X_train, y_train)
print("Class distribution after SMOTE:")
print(sorted(Counter(y_train1).items()))

# mixed oversample minority class + undersample majority class using SMOTETomek
smote = SMOTETomek(random_state=2022)
X_train2, y_train2 = smote.fit_resample(X_train, y_train)
print("Class distribution after SMOTETomek:")
print(sorted(Counter(y_train2).items()))

# mixed oversample minority class + undersample majority class using SMOTEENN
smote = SMOTEENN(random_state=2022)
X_train3, y_train3 = smote.fit_resample(X_train, y_train)
print("Class distribution after SMOTEENN:")
print(sorted(Counter(y_train3).items()))

Class distribution before resampling:
[(0, 38217), (1, 3766)]
Class distribution after SMOTE:
[(0, 38217), (1, 38217)]
Class distribution after SMOTETomek:
[(0, 38189), (1, 38189)]
Class distribution after SMOTEENN:
[(0, 21560), (1, 37809)]


In [24]:
# pickle files
data_dir = './data' # replace with your own file directory, if needed
X_train3.to_pickle('{}/X_train_PCA.pkl'.format(data_dir))     # change X_train to variable depending on chosen sampling method
X_test.to_pickle('{}/X_test_PCA.pkl'.format(data_dir))
y_train3.to_pickle('{}/y_train_PCA.pkl'.format(data_dir))
y_test.to_pickle('{}/y_test_PCA.pkl'.format(data_dir))